In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
import torch
import json
import math
from tqdm import tqdm
import numpy as np

In [2]:
max_length = 2048
adapter_path = "mgpt-lora-adapter_romania_01"
tokenizer_path = adapter_path + '_tok'

In [3]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
peft_config = PeftConfig.from_pretrained(adapter_path)

In [4]:
base_model = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path,
    torch_dtype=torch.float16,
    device_map="auto"
)

In [5]:
model = PeftModel.from_pretrained(base_model, adapter_path)
model.to("cuda")
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(100000, 2048)
        (wpe): Embedding(2048, 2048)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-23): 24 x GPT2Block(
            (ln_1): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D(nf=6144, nx=2048)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=6144, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
     

In [6]:
from datasets import load_dataset

dataset_md = load_dataset("json", data_files="moldova_dataset.json")
dataset_md = dataset_md['train'].train_test_split(test_size=0.1, seed=42)

dataset_ro = load_dataset("json", data_files="romania_dataset.json")
dataset_ro = dataset_ro['train'].train_test_split(test_size=0.1, seed=42)

In [7]:
len(dataset_md['test']), len(dataset_ro['test'])

(1274, 2305)

In [8]:
title_md = []
title_ro = []

content_md = []
content_ro = []


In [9]:
def compute_metrics(model, tokenizer, text):
    # print(text)
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(model.device)
    input_ids = inputs["input_ids"]
    
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
    
    neg_log_likelihood = outputs.loss
    perplexity = math.exp(neg_log_likelihood.item())
    
    return perplexity, neg_log_likelihood.item()

In [10]:
dataset_ro['test'][0]

{'title': 'Accident mortal, pe DN 56',
 'text': 'Două persoane au decedat în urma unui accident rutier grav care a avut loc, în noaptea de joi spre vineri, pe DN 56 în județul Dolj, în afara localității Basarabi.\n\n”În noaptea de 23/24 martie a.c., ora 01.10 polițiștii Poliției Municipiului Calafat au fost sesizați cu privire la faptul că, pe D.N 56, în afara localității Basarabi, sensul de mers către Calafat, a avut loc un accident rutier.\n\nLa fața locului s-au deplasat polițiștii care au stabilit faptul că, un bărbat, de 63 de ani, cetățean turc, în timp ce conducea o autoutilitară pe D.N. 56, în afara localității Basarabi, din direcția Maglavit către Calafat, din cauze ce urmează a fi stabilite în urma cercetărilor, a intrat în coliziune cu un autocamion stationat pe partea carosabilă, fiind defect și semnalizat corespunzător. \n\n În urma evenimentului rutier, conducătorul auto de 63 de ani și o femeie, tot de 63 de ani, cetățean bulgar, pasager în autoutilitară, au decedat.\n\n

In [11]:
for row in tqdm(dataset_ro['test']):
    # if row['title'] is None or row['title'] == '' or row['text'] is None or row['text'] == '':
    #     print(row['content'])
    #     continue
    # print(row)
    try:
        perp, negloglike = compute_metrics(model, tokenizer, row['title'])
        title_ro.append({'title': row['title'], 'perplexity': perp, 'neg_log_likelihood': negloglike})
    except:
        pass

    try:
        perp, negloglike = compute_metrics(model, tokenizer, row['text'])
        content_ro.append({'text': row['text'], 'perplexity': perp, 'neg_log_likelihood': negloglike})
    except:
        pass

100%|██████████| 2305/2305 [01:00<00:00, 38.20it/s]


In [12]:
len(title_ro), len(content_ro)

(2305, 2276)

In [13]:
for row in tqdm(dataset_md['test']):
    # if row['title'] is None or row['title'] == '' or row['text'] is None or row['text'] == '':
    #     print(row['content'])
    #     continue
    # print(row)
    try:
        perp, negloglike = compute_metrics(model, tokenizer, row['title'])
        title_md.append({'title': row['title'], 'perplexity': perp, 'neg_log_likelihood': negloglike})
    except:
        pass

    try:
        perp, negloglike = compute_metrics(model, tokenizer, row['text'])
        content_md.append({'text': row['text'], 'perplexity': perp, 'neg_log_likelihood': negloglike})
    except:
        pass

100%|██████████| 1274/1274 [00:37<00:00, 33.75it/s]


In [14]:
len(title_md), len(content_md)

(1274, 1222)

In [15]:
title_mgpt_simple_romania_01 = {"ro": title_ro, "md": title_md}
content_mgpt_simple_romania_01 = {"ro": content_ro, "md": content_md}

In [16]:
with open("title_mgpt_simple_romania_01.json", "w") as f:
    json.dump(title_mgpt_simple_romania_01, f, indent=4, ensure_ascii=False)

In [17]:
with open("content_mgpt_simple_romania_01.json", "w") as f:
    json.dump(content_mgpt_simple_romania_01, f, indent=4, ensure_ascii=False)